In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import rdkit
from rdkit import Chem

import h5py, ast, pickle

# Occupy a GPU for the model to be loaded 
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
# GPU ID, if occupied change to an available GPU ID listed under !nvidia-smi
%env CUDA_VISIBLE_DEVICES=2 

from ddc_pub import ddc_v3 as ddc

In [ ]:
def get_descriptors(smiles_list, qsar_model=None, show_actives=False, active_thresh=0.5, qed_thresh=0.5):
    """Calculate molecular descriptors of SMILES in a list.
    The descriptors are logp, tpsa, mw, qed, hba, hbd and probability of being active towards DRD2.
    
    Returns:
        A np.ndarray of descriptors.
    """
    from tqdm import tqdm_notebook as tqdm
    import rdkit
    from rdkit import Chem, DataStructs
    from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, QED
    
    descriptors = []
    active_mols = []
    
    for idx, smiles in enumerate(smiles_list):
        # Convert to mol
        mol = Chem.MolFromSmiles(smiles)
        # If valid, calculate its properties
        if mol:
            try:
                logp  = Descriptors.MolLogP(mol)
                tpsa  = Descriptors.TPSA(mol)
                molwt = Descriptors.ExactMolWt(mol)
                hba   = rdMolDescriptors.CalcNumHBA(mol)
                hbd   = rdMolDescriptors.CalcNumHBD(mol)
                qed   = QED.qed(mol)
                
                # Calculate fingerprints
                fp = AllChem.GetMorganFingerprintAsBitVect(mol,2, nBits=2048)
                ecfp4 = np.zeros((2048,))
                DataStructs.ConvertToNumpyArray(fp, ecfp4) 
                # Predict activity and pick only the second component
                active = qsar_model.predict_proba([ecfp4])[0][1]
                descriptors.append([logp, tpsa, molwt, qed, hba, hbd, active]) 
                
                if active > active_thresh and qed > qed_thresh:
                    if show_actives:
                        active_mols.append(mol)
                        print("active_proba: %.2f, QED: %.2f." % (active, qed))
                        display(mol)
                        pass
                
            except Exception as e:
                # Sanitization error: Explicit valence for atom # 17 N, 4, is greater than permitted
                print(e)
        # Else, return None
        else:
            print("Invalid generation.")
            
    return np.asarray(descriptors)

# Load QSAR model

In [ ]:
qsar_model_name = "models/qsar_model.pickle"
with open(qsar_model_name, "rb") as file:
    qsar_model = pickle.load(file)["classifier_sv"]

# Load PCB cRNN

In [ ]:
# Import existing (trained) model
# Ignore any warning(s) about training configuration or non-seriazable keyword arguments
model_name = "models/pcb_model"
model = ddc.DDC(model_name=model_name)

# Select conditions for generated molecules

In [ ]:
# Custom conditions
logp              = 3.5
tpsa              = 70.0
mw                = 350.0
qed               = 0.8
hba               = 4.0
hbd               = 1.0
drd2_active_proba = 0.9

target = np.array([logp, tpsa, mw, qed, hba, hbd, drd2_active_proba])

In [ ]:
# Convert back to SMILES
smiles_out, _ = model.predict(latent=target, temp=0) # Change temp to 1 for more funky results

# Calculate the properties of the generated structure and compare
get_descriptors(smiles_list=[smiles_out], qsar_model=qsar_model, show_actives=True)